In [1]:
# import everything you need
import numpy as np
import scipy as sp
import pandas as pd
# activate inline magics
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib.image as mpl_img
# specific fo statistics
import pingouin as pg

Define neccesary parameters

In [2]:
data_file = "FED_fMRI_fullcodesfixed_final.xls"
covariate_file = "FED_Subject_Covariates.xls"

Create the neccessary factors and sort all variables appropriately

In [3]:
# extract - data from list in .xls file(s). Outliers and all known errors have already been corrected (see recent "ALLCODESFIXED")
# read relevant content
anova = pd.read_excel(data_file, sheet_name="Sheet1",
                      usecols = ['Subject', 'Condition', 'RT'])
covariates = pd.read_excel(covariate_file, sheet_name="analysis",
                           usecols = ['FED_ID', 'BDI_22_Score', 'Gender', 'Age', 'Race'])

# remove first six subjects
anova = anova[(anova.Subject.isin(range(7, 69)))]
covariates = covariates[(covariates.FED_ID.isin(range(7, 69)))]

# create variables cue_valence and cue_validity after Condition and assign values according to trial codings (1=neg, 2=pos, 3=neu) (1=valid, 2=invalid)
neg, pos, neu = [1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]
valid, invalid = [val for val in range(7, 13)], [val for val in range(1, 7)]
# create list of valence and validity numerical indices
valence_nums, validity_nums = [1, 2, 3], [1, 2]
# copy Condition list and replace
anova['Cue_Valence'] = anova['Condition']
for valence, index in zip([neg, pos, neu], valence_nums):
    anova.loc[anova['Condition'].isin(valence), 'Cue_Valence'] = index

anova['Cue_Validity'] = anova['Condition']
for validity, index in zip([valid, invalid], validity_nums):
    anova.loc[anova['Condition'].isin(validity), 'Cue_Validity'] = index

# create variable DStatus after Subject and replace each subject value with the corresponding group identity
# separate covariate FED_IDs by BDI22
mdd = covariates.FED_ID[covariates['BDI_22_Score'] == 1]
control = covariates.FED_ID[covariates['BDI_22_Score'] == 0]
# copy Subject list and replace
anova['DStatus'] = anova['Subject']
anova.loc[anova['Subject'].isin(mdd), 'DStatus'] = 1
anova.loc[anova['Subject'].isin(control), 'DStatus'] = 0

# sort values by DStatus, valence and validity (by factor, starting with largest increment)
anova.sort_values(['DStatus', 'Cue_Valence', 'Cue_Validity'], inplace = True)

In [9]:
 pd.set_option('display.max_rows', None)
print(anova)

            RT  Subject  Condition  Cue_Valence  Cue_Validity  DStatus
1089    352.98        7          7            1             1        0
1090    309.91        7          7            1             1        0
1091    322.85        7          7            1             1        0
1092    358.01        7          7            1             1        0
1093    204.47        7          7            1             1        0
1094    330.10        7          7            1             1        0
1095    288.31        7          7            1             1        0
1096    345.30        7          7            1             1        0
1097    348.86        7          7            1             1        0
1180    308.38        7          7            1             1        0
1181    443.81        7          7            1             1        0
1182    276.96        7          7            1             1        0
1183    315.23        7          7            1             1        0
1184  

Build three-way ANOVA design and calculate the F-test

In [4]:
# check whether variances are equal using Levene tests
homoscedast_DStatus = pg.homoscedasticity(data=anova, dv='RT', group='DStatus')
homoscedast_Valence = pg.homoscedasticity(data=anova, dv='RT', group='Cue_Valence')
homoscedast_Validity = pg.homoscedasticity(data=anova, dv='RT', group='Cue_Validity')
# put all into list
homoscedast = [homoscedast_DStatus, homoscedast_Valence, homoscedast_Validity]

In [8]:
# inspect the results
for test in homoscedast:
    print(test)

                W          pval  equal_var
levene  102.77696  4.767290e-24      False
                W          pval  equal_var
levene  15.035238  3.013443e-07      False
               W      pval  equal_var
levene  4.963112  0.025913      False


In [5]:
# run the full ANOVA
FED_anova = pg.anova(data=anova, dv='RT', between=['DStatus', 'Cue_Valence', 'Cue_Validity'], ss_type=3).round(3)  # round to three place digits

In [6]:
# main effect DStatus significant, Interaction Valence-Validity too, BUT neither of their main effects! -> simple effects!^^
# fix the error variance (residual variance  from full ANOVA)
err_MS = FED_anova.MS[FED_anova.Source == 'Residual'].values
err_dof = FED_anova.DF[FED_anova.Source == 'Residual'].values

# -> simple effects as two-way ANOVA only for valid trials
simpleeff_valid = pg.anova(data=anova[anova['Cue_Validity'] == 1], dv='RT', between=['DStatus', 'Cue_Valence'], ss_type=3).round(3)  # round to three place digits
# calculate corrected F value using the residual variance  from full ANOVA
vale_MS = simpleeff_valid.MS[simpleeff_valid.Source == 'Cue_Valence'].values
vale_dof = simpleeff_valid.DF[simpleeff_valid.Source == 'Cue_Valence'].values
correctF_valence = vale_MS / err_MS
correctp_valence = 1 - sp.stats.f.cdf(correctF_valence, vale_dof, err_dof)
# insert the correct value into dataframe
simpleeff_valid.insert(6, 'F_corr', [np.nan, *correctF_valence, np.nan, np.nan])  # lose the brackets^^
simpleeff_valid.insert(7, 'p_corr', [np.nan, *correctp_valence, np.nan, np.nan])  # lose the brackets^^

# -> simple effects as two-way ANOVA only for invalid trials
simpleeff_invalid = pg.anova(data=anova[anova['Cue_Validity'] == 2], dv='RT', between=['DStatus', 'Cue_Valence'], ss_type=3).round(3)  # round to three place digits
# calculate corrected F value using the residual variance  from full ANOVA
vale_MS = simpleeff_invalid.MS[simpleeff_invalid.Source == 'Cue_Valence'].values
vale_dof = simpleeff_invalid.DF[simpleeff_invalid.Source == 'Cue_Valence'].values
correctF_valence = vale_MS / err_MS
correctp_valence = 1 - sp.stats.f.cdf(correctF_valence, vale_dof, err_dof)
# insert the correct value into dataframe
simpleeff_invalid.insert(6, 'F_corr', [np.nan, *correctF_valence, np.nan, np.nan])  # lose the brackets^^
simpleeff_invalid.insert(7, 'p_corr', [np.nan, *correctp_valence, np.nan, np.nan])  # lose the brackets^^

Plot three-way ANOVA results

In [7]:
FED_anova

,Source,SS,DF,MS,F,p-unc,np2
0,DStatus,2.989308e+05,1.0,298930.835,12.173,0.000,0.001
1,Cue_Valence,1.261261e+05,2.0,63063.037,2.568,0.077,0.000
2,Cue_Validity,1.356578e+04,1.0,13565.783,0.552,0.457,0.000
3,DStatus * Cue_Valence,1.276030e+04,2.0,6380.152,0.260,0.771,0.000
4,DStatus * Cue_Validity,2.688208e+04,1.0,26882.075,1.095,0.295,0.000
5,Cue_Valence * Cue_Validity,4.719226e+05,2.0,235961.279,9.608,0.000,0.002
6,DStatus * Cue_Valence * Cue_Validity,3.986043e+03,2.0,1993.021,0.081,0.922,0.000
7,Residual,2.737710e+08,11148.0,24557.861,NaN,NaN,NaN


In [8]:
simpleeff_valid

,Source,SS,DF,MS,F,p-unc,F_corr,p_corr,np2
0,DStatus,7.329054e+04,1,73290.544,2.872,0.090,NaN,NaN,0.001
1,Cue_Valence,5.413986e+05,2,270699.289,10.609,0.000,11.022918,0.000017,0.004
2,DStatus * Cue_Valence,2.846830e+03,2,1423.415,0.056,0.946,NaN,NaN,0.000
3,Residual,1.422220e+08,5574,25515.243,NaN,NaN,NaN,NaN,NaN


In [9]:
simpleeff_invalid

,Source,SS,DF,MS,F,p-unc,F_corr,p_corr,np2
0,DStatus,2.524560e+05,1.0,252455.959,10.697,0.001,NaN,NaN,0.002
1,Cue_Valence,6.087552e+04,2.0,30437.761,1.290,0.275,1.23943,0.289589,0.000
2,DStatus * Cue_Valence,1.383800e+04,2.0,6918.998,0.293,0.746,NaN,NaN,0.000
3,Residual,1.315491e+08,5574.0,23600.479,NaN,NaN,NaN,NaN,NaN


In [10]:
# full list of table plotting formats in pingouin:
# "plain" "simple" "github""grid" "fancy_grid" "pipe""orgtbl" "jira""presto" "pretty""psql" "rst" "mediawiki" "moinmoin" "youtrack" "html" "unsafehtml" "latex"
# "latex_raw" "latex_booktabs" "latex_longtable" "textile""tsv"
pg.print_table(FED_anova, tablefmt="latex")
pg.print_table(simpleeff_valid, tablefmt="latex")
pg.print_table(simpleeff_invalid, tablefmt="latex")


ANOVA SUMMARY

\begin{tabular}{lrrrrrr}
\hline
 Source                               &            SS &        DF &         MS &       F &   p-unc &     np2 \\
\hline
 DStatus                              &    298930.835 &     1.000 & 298930.835 &  12.173 &   0.000 &   0.001 \\
 Cue\_Valence                          &    126126.075 &     2.000 &  63063.037 &   2.568 &   0.077 &   0.000 \\
 Cue\_Validity                         &     13565.783 &     1.000 &  13565.783 &   0.552 &   0.457 &   0.000 \\
 DStatus * Cue\_Valence                &     12760.303 &     2.000 &   6380.152 &   0.260 &   0.771 &   0.000 \\
 DStatus * Cue\_Validity               &     26882.075 &     1.000 &  26882.075 &   1.095 &   0.295 &   0.000 \\
 Cue\_Valence * Cue\_Validity           &    471922.558 &     2.000 & 235961.279 &   9.608 &   0.000 &   0.002 \\
 DStatus * Cue\_Valence * Cue\_Validity &      3986.043 &     2.000 &   1993.021 &   0.081 &   0.922 &   0.000 \\
 Residual                             & 2

Calculate Enhanced cue Validity (ECV), Attentional Engagement (AE), and Difficulty in Attentional Disengagement (DAD) - subject means

In [11]:
# create RT means for smallest increment (validity)
# copy data to avoid accidents
eect_ttest = anova.copy()

# define lsit of subject numbers
fed_nums = [num for num in range(7, 69)]

# sort data by subject, valence and validity
eect_ttest.sort_values(['Subject', 'Cue_Valence', 'Cue_Validity'], inplace = True)

# list RT by subject, valence and validity, respectively
rts = {}
for sub in eect_ttest['Subject'].unique():
    for valence in eect_ttest['Cue_Valence'].unique():
        for valid in eect_ttest['Cue_Validity'].unique():
            rts[f"{sub}_{valence}_{valid}"] = eect_ttest.loc[(eect_ttest['Subject'] == sub)
                                                           & (eect_ttest['Cue_Valence'] == valence) 
                                                           & (eect_ttest['Cue_Validity'] == valid), 'RT'].tolist()

In [12]:
# calculate ECV for each valence per subject
mean_ecv = {}
# start at subject index level
for sub in fed_nums:
    # iterate across valences
    for val in valence_nums:
        # create list for validity-referenced combos
        validity = []
        # go through combinations of conditions by subject
        for combo in rts.keys():
            # fix the subject ID
            if (int(combo.split('_', 1)[0]) == sub and int(combo.split('_', 2)[1]) == val):
                # collect validities
                validity.append(combo)
        # calculate the mean of the inv-v difference by valence
        mean_ecv[f"{sub}_{val}"] = np.mean([inv - v for inv, v in zip(rts[validity[1]], rts[validity[0]])])

# transfer to pd.dataframe
mean_ecv = (pd.DataFrame.from_dict(mean_ecv, orient='index', columns=['mean_ECV']).rename_axis('FEDbycondition').reset_index())

In [13]:
# calculate AE for each valence per subject
mean_ae = {}
# start at subject index level
for sub in fed_nums:
    # iterate across valences
    ref = 3  # neutral is always reference
    for val in validity_nums:
        # go through combinations of conditions by subject
        for combo in rts.keys():
            # set valid neutral reference
            reference = f"{sub}_{ref}_{1}"
            # fix the subject ID, and validity to 1 (valid)
            if (int(combo.split('_', 1)[0]) == sub and int(combo.split('_', 2)[1]) == val and int(combo.rsplit('_', 1)[1]) == 1):
                # calculate the mean of the vneu-vemo difference by valence
                mean_ae[f"{sub}_{val}"] = np.mean([vneu - vemo for vneu, vemo in zip(rts[reference], rts[combo])])

# transfer to pd.dataframe
mean_ae = (pd.DataFrame.from_dict(mean_ae, orient='index', columns=['mean_AE']).rename_axis('FEDbycondition').reset_index())

In [14]:
# calculate DAD for each valence per subject
mean_dad = {}
# start at subject index level
for sub in fed_nums:
    # iterate across valences
    ref = 3  # neutral is always reference
    for val in validity_nums:
        # go through combinations of conditions by subject
        for combo in rts.keys():
            # set invalid neutral reference
            reference = f"{sub}_{ref}_{2}"
            # fix the subject ID, and validity to 2 (invalid)
            if (int(combo.split('_', 1)[0]) == sub and int(combo.split('_', 2)[1]) == val and int(combo.rsplit('_', 1)[1]) == 2):
                # calculate the mean of the invemo-invneu difference by valence
                mean_dad[f"{sub}_{val}"] = np.mean([invemo - invneu for invemo, invneu in zip(rts[combo], rts[reference])])

# transfer to pd.dataframe
mean_dad = (pd.DataFrame.from_dict(mean_dad, orient='index', columns=['mean_DAD']).rename_axis('FEDbycondition').reset_index())

Inspect and make sure that all variables look OK

In [25]:
print(mean_dad)

    FEDbycondition   mean_DAD  FED  DStatus
0              7_1 -24.264138    7        0
1              7_2   9.287000    7        0
2              8_1   8.107931    8        0
3              8_2 -11.112000    8        0
4              9_1  31.534828    9        0
..             ...        ...  ...      ...
101           57_2  20.808667   57        1
104           59_1  -0.530690   59        1
105           59_2 -34.291333   59        1
118           66_1  12.008966   66        1
119           66_2  10.141667   66        1

[124 rows x 4 columns]


Calculate two-sided t-tests (un)like Koster (2005)

In [16]:
# add group column to each variable's dataframe
for frame in [mean_ecv, mean_ae, mean_dad]:
    # copy Subject list and transform to integers
    frame['FED'] = frame['FEDbycondition'].apply(lambda x: int(x.split('_', 1)[0]))
    # copy again to obtain DStatus list
    frame['DStatus'] = frame['FED']
    # check and change by group
    frame.loc[frame['FED'].isin(mdd), 'DStatus'] = 1
    frame.loc[frame['FED'].isin(control), 'DStatus'] = 0
    # finally, sort the dataframe by DStatus and FED
    frame.sort_values(['DStatus', 'FED'], inplace = True)

In [17]:
# calculate cauchy scaling factor of Koster et al. (2005)
# - original: https://doi.org/10.3758/PBR.16.2.225; recent implimentation: https://doi.org/10.1080/00031305.2018.1562983

# transfer t_vals to cohens d for basing on propability density function
smallest_t = 2.15
largest_t = 2.72
small_cohen = pg.compute_effsize_from_t(smallest_t, nx=20, ny=20, eftype='cohen')
large_cohen = pg.compute_effsize_from_t(largest_t, N=15, eftype='cohen')
# spectrum based on Exp1 & Exp2 results
# t-values are between 0 and 1.4, round up
value_halfwidth = 2
# set scaling factor required for estimated 80% probability that the true effect sizes lie within the spectrum (+2 - -2)
cauchy_r = 0.707
# control the probability
effect_prob_est = sp.stats.cauchy.cdf(value_halfwidth, 0, cauchy_r) - sp.stats.cauchy.cdf(-value_halfwidth, 0, cauchy_r)

In [47]:
# run a two-sided t-test with appropriate p-level threshold (unlike Koster ...)
# for ECV
# neg
ecvneg_mdd = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_ecv['DStatus'] == 1)].tolist()
ecvneg_con = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_ecv['DStatus'] == 0)].tolist()
# pos
ecvpos_mdd = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_ecv['DStatus'] == 1)].tolist()
ecvpos_con = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_ecv['DStatus'] == 0)].tolist()
# neu
ecvneu_mdd = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}3$')) & (mean_ecv['DStatus'] == 1)].tolist()
ecvneu_con = mean_ecv['mean_ECV'][(mean_ecv['FEDbycondition'].str.contains('.{2,3}3$')) & (mean_ecv['DStatus'] == 0)].tolist()
# put all into tupled list
ecv_testbtw = [(ecvneg_mdd, ecvneg_con), (ecvpos_mdd, ecvpos_con), (ecvneu_mdd, ecvneu_con)]
ecv_testwithin = [[group[1] for group in ecv_testbtw], [group[0] for group in ecv_testbtw]]

# test all valences separately between and within group
# build empty list to hold all tests
ttests_ecv_all = []
# first compare valences between groups with a 2-sample t-test
for valence in ecv_testbtw:
    # index + 1 to keep valence numbering consistent
    ttests_ecv = pg.ttest(valence[0], valence[1], paired=False, tail='two-sided', r=cauchy_r, confidence=0.95)
    # add condition ID
    ttests_ecv.insert(0, "ConditionID", f"ecv_btw_{ecv_testbtw.index(valence) + 1}")
    # add (partial) eta-squared estimate to the dataframe
    ttests_ecv.insert(7, "eta-squared", pg.compute_effsize(valence[0], valence[1], paired=False, eftype='eta-square'))
    # put the finish dataframe into list
    ttests_ecv_all.append(ttests_ecv)

# Now look at the valences within groups with a paired t-test
for group in ecv_testwithin:
    # compare neg and pos against neu
    for valence in [0, 1]:
        # index + 1 to keep valence numbering consistent
        ttests_ecv = pg.ttest(group[valence], group[2], paired=True, tail='two-sided', r=cauchy_r, confidence=0.95)
        # add condition and group ID (0 means control^^)
        ttests_ecv.insert(0, "ConditionID", f"ecv_within_{ecv_testwithin.index(group)}_{valence + 1}")
        # add (partial) eta-squared estimate to the dataframe
        ttests_ecv.insert(7, "eta-squared", pg.compute_effsize(group[valence], group[2], paired=True, eftype='eta-square'))
        # put the finish dataframe into list
        ttests_ecv_all.append(ttests_ecv)

# transfer list to dataframe
ttests_ecv_all = pd.concat(ttests_ecv_all)

In [41]:
# test whether or not all t-test RT lists have the same length
for group in ecv_testwithin:
    print(f"Negative ECV {len(group[0])}; Positive ECV {len(group[1])}; Neutral ECV {len(group[2])}")

Negative ECV 31; Positive ECV 31; Neutral ECV 31
Negative ECV 31; Positive ECV 31; Neutral ECV 31


In [42]:
# run a two-sided t-test with appropriate p-level threshold (unlike Koster ...)
# for AE
# neg
aeneg_mdd = mean_ae['mean_AE'][(mean_ae['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_ae['DStatus'] == 1)].tolist()
aeneg_con = mean_ae['mean_AE'][(mean_ae['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_ae['DStatus'] == 0)].tolist()
# pos
aepos_mdd = mean_ae['mean_AE'][(mean_ae['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_ae['DStatus'] == 1)].tolist()
aepos_con = mean_ae['mean_AE'][(mean_ae['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_ae['DStatus'] == 0)].tolist()
# put all into tupled list (no within comparisons here)
ae_testbtw = [(aeneg_mdd, aeneg_con), (aepos_mdd, aepos_con)]

# test all valences separately between and within group
# build empty list to hold all tests
ttests_ae_all = []
# first compare valences between groups with a 2-sample t-test
for valence in ae_testbtw:
    # index + 1 to keep valence numbering consistent
    ttests_ae = pg.ttest(valence[0], valence[1], paired=False, tail='two-sided', r=cauchy_r, confidence=0.95)
    # add condition ID
    ttests_ae.insert(0, "ConditionID", f"ae_btw_{ae_testbtw.index(valence) + 1}")
    # add (partial) eta-squared estimate to the dataframe
    ttests_ae.insert(7, "eta-squared", pg.compute_effsize(valence[0], valence[1], paired=False, eftype='eta-square'))
    # put the finish dataframe into list
    ttests_ae_all.append(ttests_ae)

# transfer list to dataframe
ttests_ae_all = pd.concat(ttests_ae_all)

In [43]:
# run a two-sided t-test with appropriate p-level threshold (unlike Koster ...)
# for DAD
# neg
dadneg_mdd = mean_dad['mean_DAD'][(mean_dad['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_dad['DStatus'] == 1)].tolist()
dadneg_con = mean_dad['mean_DAD'][(mean_dad['FEDbycondition'].str.contains('.{2,3}1$')) & (mean_dad['DStatus'] == 0)].tolist()
# pos
dadpos_mdd = mean_dad['mean_DAD'][(mean_dad['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_dad['DStatus'] == 1)].tolist()
dadpos_con = mean_dad['mean_DAD'][(mean_dad['FEDbycondition'].str.contains('.{2,3}2$')) & (mean_dad['DStatus'] == 0)].tolist()
# put all into tupled list (no within comparisons here)
dad_testbtw = [(dadneg_mdd, dadneg_con), (dadpos_mdd, dadpos_con)]

# test all valences separately between and within group
# build empty list to hold all tests
ttests_dad_all = []
# first compare valences between groups with a 2-sample t-test
for valence in dad_testbtw:
    # index + 1 to keep valence numbering consistent
    ttests_dad = pg.ttest(valence[0], valence[1], paired=False, tail='two-sided', r=cauchy_r, confidence=0.95)
    # add condition ID
    ttests_dad.insert(0, "ConditionID", f"dad_btw_{dad_testbtw.index(valence) + 1}")
    # add (partial) eta-squared estimate to the dataframe
    ttests_dad.insert(7, "eta-squared", pg.compute_effsize(valence[0], valence[1], paired=False, eftype='eta-square'))
    # put the finish dataframe into list
    ttests_dad_all.append(ttests_dad)

# transfer list to dataframe
ttests_dad_all = pd.concat(ttests_dad_all)

Plot t-test results

In [48]:
# view respective ttest dicts to make sure everything looks as it should
ttests_ecv_all

,ConditionID,T,dof,tail,p-val,CI95%,cohen-d,eta-squared,BF10,power
T-test,ecv_btw_1,-0.190681,60,two-sided,0.849419,"[-24.24, 20.02]",0.048433,0.000586,0.263,0.054043
T-test,ecv_btw_2,-1.097178,60,two-sided,0.276948,"[-24.21, 7.06]",0.278683,0.019046,0.43,0.190535
T-test,ecv_btw_3,-0.776735,60,two-sided,0.440366,"[-20.77, 9.15]",0.197291,0.009637,0.334,0.119147
T-test,ecv_within_0_1,4.100952,30,two-sided,0.000289,"[11.92, 35.58]",0.836257,0.148814,99.117,0.994498
T-test,ecv_within_0_2,-0.601208,30,two-sided,0.552218,"[-11.35, 6.19]",0.103956,0.002694,0.226,0.086694
T-test,ecv_within_1_1,2.901146,30,two-sided,0.006900,"[8.13, 46.78]",0.620390,0.087775,6.109,0.916467
T-test,ecv_within_1_2,-0.801471,30,two-sided,0.429162,"[-18.97, 8.28]",0.154526,0.005934,0.257,0.132504


In [49]:
ttests_ae_all

,ConditionID,T,dof,tail,p-val,CI95%,cohen-d,eta-squared,BF10,power
T-test,ae_btw_1,-0.467297,60,two-sided,0.641980,"[-18.44, 11.46]",0.118694,0.003510,0.284,0.074558
T-test,ae_btw_2,-0.429936,60,two-sided,0.668782,"[-10.93, 7.06]",0.109204,0.002973,0.28,0.070747


In [50]:
ttests_dad_all

,ConditionID,T,dof,tail,p-val,CI95%,cohen-d,eta-squared,BF10,power
T-test,dad_btw_1,0.498991,60,two-sided,0.619610,"[-14.67, 24.42]",0.126744,0.004000,0.288,0.078051
T-test,dad_btw_2,-0.110757,60,two-sided,0.912178,"[-15.81, 14.15]",0.028132,0.000198,0.26,0.051362


In [54]:
# full list of table plotting formats in pingouin:
# "plain" "simple" "github""grid" "fancy_grid" "pipe""orgtbl" "jira""presto" "pretty""psql" "rst" "mediawiki" "moinmoin" "youtrack" "html" "unsafehtml" "latex"
# "latex_raw" "latex_booktabs" "latex_longtable" "textile""tsv"
pg.print_table(ttests_ecv_all, tablefmt="latex")
pg.print_table(ttests_ae_all, tablefmt="latex")
pg.print_table(ttests_dad_all, tablefmt="latex")

\begin{tabular}{lrrlrlrrrr}
\hline
 ConditionID    &      T &   dof & tail      &   p-val & CI95\%           &   cohen-d &   eta-squared &   BF10 &   power \\
\hline
 ecv\_btw\_1      & -0.191 &    60 & two-sided &   0.849 & [-24.24  20.02] &     0.048 &         0.001 &  0.263 &   0.054 \\
 ecv\_btw\_2      & -1.097 &    60 & two-sided &   0.277 & [-24.21   7.06] &     0.279 &         0.019 &  0.430 &   0.191 \\
 ecv\_btw\_3      & -0.777 &    60 & two-sided &   0.440 & [-20.77   9.15] &     0.197 &         0.010 &  0.334 &   0.119 \\
 ecv\_within\_0\_1 &  4.101 &    30 & two-sided &   0.000 & [11.92 35.58]   &     0.836 &         0.149 & 99.117 &   0.994 \\
 ecv\_within\_0\_2 & -0.601 &    30 & two-sided &   0.552 & [-11.35   6.19] &     0.104 &         0.003 &  0.226 &   0.087 \\
 ecv\_within\_1\_1 &  2.901 &    30 & two-sided &   0.007 & [ 8.13 46.78]   &     0.620 &         0.088 &  6.109 &   0.916 \\
 ecv\_within\_1\_2 & -0.801 &    30 & two-sided &   0.429 & [-18.97   8.28] &    

Build summary tables for each variable by group, valence (and validity, if needed)

In [ ]:
# collect the cohort characteristics by calculating descriptives for each variable between groups



MDD_mean_ecv_neg = 
MDD_mean_ecv_pos = 
MDD_mean_ecv_neu = 
control_mean_ecv_neg = 
control_mean_ecv_pos = 
control_mean_ecv_neu = 
MDD_mean_ae_neg = 
MDD_mean_ae_pos = 
MDD_mean_dad_neg = 
MDD_mean_dad_pos = 
control_mean_ae_neg = 
control_mean_ae_pos = 
control_mean_dad_neg = 
control_mean_dad_pos = 

# Testing Zone

In [ ]:


# # transfer to pd.dataframe
# rts_pd = (pd.DataFrame.from_dict(rts, orient='index', columns=['mean_RT']).rename_axis('condition').reset_index())